In [1]:
import pandas as pd
import os

In [2]:
overview = pd.read_csv(os.path.join('results','overview.csv'), sep=';')
overview

,run_id,gs_id,duration,fitness_mean,fitness_sd,stopped_early,co_p,crossover,difficulty,diversity_measure,...,fitness_sharing,gens,mu_p,mutation,optim,pop_size,representation,selection,user_id,comments
0,0,0,83.06,92.07,2.22,30,0.7,single_point_co,3,True,...,False,1000,0.05,swap_mutation,min,100,with_replacement,tournament,dp,NaN
1,0,1,115.95,92.80,1.83,30,0.7,single_point_co,3,True,...,False,1000,0.05,swap_mutation,min,100,with_replacement,fps,dp,NaN
2,0,2,78.09,88.20,1.87,30,0.7,single_point_co,3,True,...,False,1000,0.05,swap_mutation,min,100,without_replacement,tournament,dp,NaN
3,0,3,126.25,88.07,1.59,30,0.7,single_point_co,3,True,...,False,1000,0.05,swap_mutation,min,100,without_replacement,fps,dp,NaN
4,0,4,147.44,25.97,1.40,30,0.7,single_point_co,3,True,...,False,1000,0.05,swap_mutation,min,100,maintain_init_puzzle,tournament,dp,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,23,187,0.61,29.50,1.50,2,0.5,single_point_co,3,True,...,False,100,0.90,swap_mutation,min,30,maintain_init_puzzle,tournament,dp,NaN
716,23,188,0.41,29.50,1.50,2,0.5,single_point_co,3,True,...,False,100,0.90,inversion_mutation,min,30,random_mix,tournament,dp,NaN
717,23,189,0.23,324.00,5.00,2,0.5,single_point_co,3,True,...,False,100,0.90,inversion_mutation,min,30,with_replacement,tournament,dp,NaN
718,23,190,0.30,318.00,14.00,2,0.5,single_point_co,3,True,...,False,100,0.90,inversion_mutation,min,30,without_replacement,tournament,dp,NaN


In [3]:
overview.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 720 entries, 0 to 719
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   run_id                   720 non-null    int64  
 1   gs_id                    720 non-null    int64  
 2   duration                 720 non-null    float64
 3   fitness_mean             720 non-null    float64
 4   fitness_sd               720 non-null    float64
 5   stopped_early            720 non-null    int64  
 6   co_p                     720 non-null    float64
 7   crossover                720 non-null    object 
 8   difficulty               720 non-null    int64  
 9   diversity_measure        720 non-null    bool   
 10  early_stopping_patience  720 non-null    int64  
 11  elitism                  720 non-null    bool   
 12  epochs                   720 non-null    int64  
 13  fitness_sharing          720 non-null    bool   
 14  gens                     7

In [9]:
overview['fitness_mean']

0       92.07
1       92.80
2       88.20
3       88.07
4       25.97
        ...  
715     29.50
716     29.50
717    324.00
718    318.00
719     28.50
Name: fitness_mean, Length: 720, dtype: float64

In [17]:
overview.loc[overview.run_id == 23, ].sort_values(by='fitness_mean').head(10)

,run_id,gs_id,duration,fitness_mean,fitness_sd,stopped_early,co_p,crossover,difficulty,diversity_measure,...,fitness_sharing,gens,mu_p,mutation,optim,pop_size,representation,selection,user_id,comments
707,23,179,1.29,27.0,2.0,2,0.50,single_point_co,3,True,...,False,100,0.80,swap_mutation,min,30,maintain_init_puzzle,tournament,dp,NaN
532,23,4,2.67,27.0,1.0,2,0.95,single_point_co,3,True,...,True,100,0.05,inversion_mutation,min,30,random_mix,tournament,dp,NaN
543,23,15,4.25,27.0,4.0,2,0.95,single_point_co,3,True,...,True,100,0.20,inversion_mutation,min,30,maintain_init_puzzle,tournament,dp,NaN
652,23,124,0.42,27.5,0.5,2,0.85,single_point_co,3,True,...,False,100,0.90,inversion_mutation,min,30,random_mix,tournament,dp,NaN
704,23,176,0.54,27.5,2.5,2,0.50,single_point_co,3,True,...,False,100,0.80,swap_mutation,min,30,random_mix,tournament,dp,NaN
575,23,47,0.85,27.5,0.5,2,0.95,single_point_co,3,True,...,False,100,0.20,inversion_mutation,min,30,maintain_init_puzzle,tournament,dp,NaN
547,23,19,3.60,27.5,0.5,2,0.95,single_point_co,3,True,...,True,100,0.80,swap_mutation,min,30,maintain_init_puzzle,tournament,dp,NaN
583,23,55,1.34,27.5,0.5,2,0.95,single_point_co,3,True,...,False,100,0.80,inversion_mutation,min,30,maintain_init_puzzle,tournament,dp,NaN
535,23,7,3.00,27.5,0.5,2,0.95,single_point_co,3,True,...,True,100,0.05,inversion_mutation,min,30,maintain_init_puzzle,tournament,dp,NaN
679,23,151,2.20,27.5,0.5,2,0.50,single_point_co,3,True,...,True,100,0.80,inversion_mutation,min,30,maintain_init_puzzle,tournament,dp,NaN
